In [1]:
from os import listdir
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib
from itertools import product

model_path = f".\\models\\image_classifier_{len(listdir('models')) - 1}"

%matplotlib inline


In [ ]:
# Load and normalize CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./datasets', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print(f"\nTraining on {len(trainset)} images.\nTesting on {len(testset)} images.")

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def get_feature_maps(self, x):
        feature_maps = []
        x = F.relu(self.conv1(x))
        feature_maps.append(x)
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        feature_maps.append(x)
        x = self.pool(x)
        return feature_maps

    def visualize_feature_maps(self, feature_maps):
        for i, feature_map in enumerate(feature_maps):
            plt.figure(figsize=(15, 15))
            for j in range(feature_map.shape[1]):
                plt.subplot(1, feature_map.shape[1], j+1)
                plt.imshow(feature_map[0, j].detach().cpu().numpy(), cmap='gray')
                plt.axis('off')
            plt.show()

    def get_gradient(self, x, y):
        # Ensure the input requires gradient
        x.requires_grad = True

        output = self.forward(x)

        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, y)

        # Calculate gradient
        self.zero_grad()
        loss.backward()
        gradient = x.grad.data

        return gradient

    def visualize_input_and_gradient(self, x, gradient):
        # Convert tensors to numpy arrays
        img = x[0].detach().cpu().numpy().transpose(1, 2, 0)
        gradient_arr = gradient[0].detach().cpu().numpy().transpose(1, 2, 0)

        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.title("Input Image")
        plt.imshow(img)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("Gradient")
        plt.imshow(gradient_arr, cmap='gray')
        plt.axis('off')

net = Net()
# net = joblib.load("models\\image_classifier_2")


In [ ]:
# Find best hyperparameters

find_best_hyperparams = False

best_params = {'learning_rate': 0.001, 
               'batch_size': 64, 
               'epochs': 50,
               'weight_decay': 0.0001}


if find_best_hyperparams:

    # Hyperparameters
    learning_rates = [0.08, 0.1, 0.12]
    batch_sizes = [32, 64, 128]
    epochs = [1]

    best_accuracy = 0
    best_params = {}
    for lr, batch_size, epoch in product(learning_rates, batch_sizes, epochs):
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
        
        net = Net()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
        
        # Train model
        for e in range(epoch):
            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

                if i % 5000 == 4999:
                    print(f'[Epoch {epoch + 1}, Batch {i + 1}]\tLoss: {running_loss / 5000:.3f}')
                    running_loss = 0.0

        # Test model
        all_labels = []
        all_predictions = []
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs, 1)
                all_labels.extend(labels.numpy())
                all_predictions.extend(predicted.numpy())
        
        accuracy = accuracy_score(all_labels, all_predictions)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'learning_rate': lr, 'batch_size': batch_size, 'epochs': epoch}

    print(f'Best Accuracy: {best_accuracy:.3f}')
    print(f'Best Hyperparameters: {best_params}')


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr= best_params['learning_rate'], weight_decay= best_params['weight_decay'])

# optimizer = optim.SGD(net.parameters(), lr= best_params['learning_rate'], wheightmomentum=0.9)

trainloader = torch.utils.data.DataLoader(trainset, batch_size= best_params['batch_size'], shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size= best_params['batch_size'], shuffle=False)

# Train network
for epoch in range(best_params['epochs']):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, label = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}]\tLoss: {running_loss / 5000:.3f}')
            running_loss = 0.0

print('Finished Training')


In [ ]:

all_labels = []
all_predictions = []

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')


In [ ]:
# Plot one classification for each label in the same figure
label_found = [False] * 10
dataiter = iter(testloader)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

while not all(label_found):
    images, labels = next(dataiter)
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)

    for i in range(len(labels)):
        label = labels[i].item()
        if not label_found[label]:
            ax = axes[label // 5, label % 5]
            ax.imshow(images[i].permute(1, 2, 0) / 2 + 0.5)  # unnormalize
            ax.set_title(f'Predicted: {classes[predicted[i]]}\nActual: {classes[label]}')
            ax.axis('off')
            label_found[label] = True

plt.tight_layout()


In [ ]:
model_path = f".\\models\\image_classifier_{len(listdir('models'))}"
joblib.dump(net, model_path)
